# <center>Phoenix in Flight</center>
## <center>Investigating Embedding Drift for a Sentiment Classification Model</center>

Imagine you're in charge of maintaining a model that takes as input online reviews of your U.S.-based product and classifies the sentiment of each review as positive, negative, or neutral. Your model initially performs well in production, but its performance gradually degrades over time.

Phoenix helps you surface the reason for this regression by analyzing the *embeddings* representing the text of each review. Your model was trained on English reviews, but as you'll discover, it's encountering Spanish reviews in production that it can't correctly classify.

According to our research, embedding drift often precedes performance degradation. So Phoenix can help you proactively detect and fix this issue before it becomes noticable to your users.

In this tutorial, you will:
* Download curated datasets of embeddings and predictions
* Visually explore embeddings in Phoenix
* Investigate problematic clusters
* Export data for labeling and re-training

Let's get started!

### 1. Install Dependencies and Import Libraries 📚

In [1]:
# %pip install -q arize-phoenix

import pandas as pd
import phoenix as px
from phoenix.datasets import Dataset, EmbeddingColumnNames, Schema

### 2. Download the Data 📊

Load your training and production data into two pandas dataframes and inspect a few rows of the training dataframe.

In [2]:
training_dataframe = pd.read_parquet(
    "https://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/nlp/sentiment-classification-language-drift/sentiment_classification_language_drift_training.parquet",
)
production_dataframe = pd.read_parquet(
    "https://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/nlp/sentiment-classification-language-drift/sentiment_classification_language_drift_production.parquet",
)
training_dataframe.head()

,prediction_ts,reviewer_age,reviewer_gender,product_category,language,text,text_vector,label,pred_label
0,1.650092e+09,21,female,apparel,english,Poor quality of fabric and ridiculously tight ...,"[-0.070516996, 0.6640034, 0.33579218, -0.26907...",negative,negative
1,1.650093e+09,29,male,kitchen,english,"Love these glasses, thought they'd be everyday...","[-0.0024410924, -0.5406275, 0.31713492, -0.033...",positive,positive
2,1.650093e+09,26,female,sports,english,"These are disgusting, it tastes like you are ""...","[0.40487882, 0.8235396, 0.38333943, -0.4269158...",negative,negative
3,1.650093e+09,26,male,other,english,My husband has a pair of TaoTronics so I decid...,"[0.018816521, 0.53441304, 0.4907303, -0.024163...",neutral,neutral
4,1.650093e+09,37,male,home_improvement,english,"Threads too deep. Engages on tank, but gasket ...","[-0.25348073, 0.31603432, 0.35810202, -0.24672...",negative,negative


The columns of the dataframe are:
- **prediction_ts:** the Unix timestamps of your predictions
- **review_age**, **reviewer_gender**, **product_category**, **language:** the features of your model
- **text:** the text of each review
- **text_vector:** the embedding vectors representing each review
- **pred_label:** the label your model predicted
- **label:** the ground-truth label for each review

### 3. Launch Phoenix 🔥🐦

#### a) Define Your Schema

To launch Phoenix with your data, you first need to define a schema that tells Phoenix which columns of your dataframes correspond to features, predictions, actuals (i.e., ground truth), embeddings, etc.

The trickiest part is defining embedding features. In this case, each embedding feature has two pieces of information: the embedding vector itself contained in the "text_vector" column and the review text contained in the "text" column.

In [3]:
embedding_features = {
    "text_embedding": EmbeddingColumnNames(
        vector_column_name="text_vector", raw_data_column_name="text"
    ),
}
schema = Schema(
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="pred_label",
    actual_label_column_name="label",
    embedding_feature_column_names=embedding_features,
)

You'll notice that the schema above doesn't explicitly specify features. That's because feature columns are automatically inferred if you don't pass `feature_column_names` to your `Schema` object.

#### b) Define Your Datasets 
Next, define your primary and reference datasets. In this case, your reference dataset contains training data and your primary dataset contains production data.

In [4]:
primary_dataset = Dataset(dataframe=production_dataframe, schema=schema, name="primary")
reference_dataset = Dataset(dataframe=training_dataframe, schema=schema, name="reference")

Dataset info written to '/Users/mikeldking/.phoenix/datasets/primary'
Dataset already persisted
Dataset: primary initialized
Dataset info written to '/Users/mikeldking/.phoenix/datasets/reference'
Dataset already persisted
Dataset: reference initialized


#### c) Create a Phoenix Session

In [5]:
session = px.launch_app(primary=primary_dataset, reference=reference_dataset)

#### d) Launch the Phoenix UI

In [7]:
session.view()

Starting Phoenix App
            primary dataset: primary
            reference dataset: reference
INFO:     127.0.0.1:59743 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:59743 - "GET /index.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:59743 - "POST /graphql HTTP/1.1" 200 OK
INFO:     127.0.0.1:59743 - "POST /graphql HTTP/1.1" 200 OK
node: EmbeddingDimension 0
INFO:     127.0.0.1:59743 - "POST /graphql HTTP/1.1" 200 OK
node: EmbeddingDimension 0
INFO:     127.0.0.1:59743 - "POST /graphql HTTP/1.1" 200 OK


Invalid comparison between dtype=datetime64[ns] and Timestamp

GraphQL request:9:7
 8 |     ... on EmbeddingDimension {
 9 |       driftTimeSeries(metric: euclideanDistance, timeRange: $timeRange) {
   |       ^
10 |         data {
Traceback (most recent call last):
  File "/Users/mikeldking/.virtualenvs/phoenix/lib/python3.10/site-packages/pandas/core/arrays/datetimelike.py", line 582, in _validate_comparison_value
    self._check_compatible_with(other)
  File "/Users/mikeldking/.virtualenvs/phoenix/lib/python3.10/site-packages/pandas/core/arrays/datetimes.py", line 461, in _check_compatible_with
    self._assert_tzawareness_compat(other)
  File "/Users/mikeldking/.virtualenvs/phoenix/lib/python3.10/site-packages/pandas/core/arrays/datetimes.py", line 694, in _assert_tzawareness_compat
    raise TypeError(
TypeError: Cannot compare tz-naive and tz-aware datetime-like objects.

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  Fi

node: EmbeddingDimension 0
INFO:     127.0.0.1:59766 - "POST /graphql HTTP/1.1" 200 OK


### 4. Explore Your Data 📈

Phoenix is under active development. At the moment, we display your model schema and a few data quality statistics. Check back soon for more updates.

### 5. Close the App 🧹

When you're done, don't forget to close the app.

In [ ]:
px.close_app()